<a href="https://colab.research.google.com/github/trentfridey/qosf-screening/blob/master/Task1FromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import autograd.numpy as np
from autograd import grad
from functools import reduce
from autograd.misc.optimizers import adam
import matplotlib.pyplot as plt

In [22]:
# Operators

sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])

p0 = np.array([[1,0],[0,0]])
p1 = np.array([[0,0],[0,1]])

# Gates

def Rx(theta):
  mat = -1j*theta*0.5*sx
  return np.exp(mat)

def Ry(theta):
  mat = -1j*theta*0.5*sy
  return np.exp(mat)

def Rz(theta):
  return np.array([[np.exp(-0.5j*theta), 0], [0, np.exp(0.5j*theta)]])

def CZ(i,j):
  ops0 = [np.eye(2) for i in range(4)]
  ops1 = [np.eye(2) for i in range(4)]
  ops0[i] = p0
  ops1[i] = p1
  ops1[j] = sz
  cz0 = reduce(lambda res, op: np.kron(res, op), ops0)
  cz1 = reduce(lambda res, op: np.kron(res, op), ops1)
  return cz0 + cz1

# Blocks

def odd(thetas):
  rots = [Rx(theta) for theta in thetas[1:]]
  rot = Rx(thetas[0])
  for r in rots:
    rot = np.kron(rot,r)
  return rot

def even(thetas):
  rots = [Rz(theta) for theta in thetas[1:]]
  rot = Rx(thetas[0])
  for r in rots:
    rot = np.kron(rot, r)
  CZs = np.eye(16)
  for i in range(3):
    for j in range(i+1, 4):
      CZs = CZs @ CZ(i,j)
  return rot @ CZs

# Layer

def layer(thetas):
  return odd(thetas[0:4]) @ even(thetas[4:8])

n_layers = 1

# Circuit

def circuit(thetas):
  circuit = np.eye(16)
  for l in range(n_layers):
    circuit = circuit @ layer(thetas[8*l:8*(l+1)])
  return circuit

# Objective

target = np.array([0 for i in range(0,15)] + [1])

def objective(thetas,iter):
  start_state = np.array([1] + [0 for i in range(1,16)])
  circ = circuit(thetas)
  return np.sum(np.abs(np.dot(circ, start_state) - target))

# Optimization

grad_obj = grad(objective)

init_thetas = np.random.normal(0, np.pi, 16)
step_size = 0.1
num_iters = 200

def handle_step(params, iter, grad):
  if iter % 10 == 0:
    print("Cost after {} steps is {}".format(iter, objective(params, iter)))

optimized_thetas = adam(grad_obj, init_thetas, step_size=step_size, num_iters=num_iters, callback=handle_step)

epsilon = objective(optimized_thetas,0)

Cost after 0 steps is 14.152414044109356
Cost after 10 steps is 11.31426865659585
Cost after 20 steps is 6.874159291369494
Cost after 30 steps is 1.075265191124774
Cost after 40 steps is 2.619912167267409
Cost after 50 steps is 1.9145664570655883
Cost after 60 steps is 1.4909038772663332
Cost after 70 steps is 1.2039055541992592
Cost after 80 steps is 1.0840711056458465
Cost after 90 steps is 1.1639905922984592
Cost after 100 steps is 1.0417035183650534
Cost after 110 steps is 1.0304719437084184
Cost after 120 steps is 1.1425541669894048
Cost after 130 steps is 1.256811605739923
Cost after 140 steps is 1.2758932896988693
Cost after 150 steps is 1.167154690130583
Cost after 160 steps is 1.0514719318987054
Cost after 170 steps is 1.1672956149469174
Cost after 180 steps is 1.08882257385395
Cost after 190 steps is 1.1886710115710726
